<a href="https://colab.research.google.com/github/markchiang/FacialScores/blob/main/FacialScores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/markchiang/FacialScores.git
!cp -R FacialScores/?? ./

Cloning into 'FacialScores'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), 88.23 KiB | 4.01 MiB/s, done.


In [3]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Define function to load images and labels from folders
def load_labeled_dataset():
    images = []
    labels = []
    for age in range(20, 80, 10):
        folder_path = f'{age}'
        image_filenames = os.listdir(folder_path)
        for image_filename in image_filenames:
          if os.path.splitext(image_filename)[1].lower() in ('.jpg', '.jpeg', '.png', '.bmp'):
            image_path = os.path.join(folder_path, image_filename)
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (128, 128), interpolation=cv2.INTER_AREA)
            images.append(resized)
            labels.append(age)
    return np.array(images), np.array(labels)

# Preprocess images for model input
def preprocess_images(images):
    normalized = images / 255.0
    batched = np.expand_dims(normalized, axis=-1)
    return batched

# Load dataset of labeled images
images, labels = load_labeled_dataset()

# Preprocess images for model input
preprocessed_images = preprocess_images(images)

# Define model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile model with loss function and optimizer
model.compile(loss='mse', optimizer=optimizers.Adam(lr=0.001))

# Train model on labeled dataset
model.fit(preprocessed_images, labels, epochs=100, batch_size=32)

# Save model for future use
model.save('facial_condition_model.h5')

# Load pre-trained model
model = tf.keras.models.load_model('facial_condition_model.h5')

# Function to preprocess image for model input
def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize image to 128x128 pixels
    resized = cv2.resize(gray, (128, 128), interpolation=cv2.INTER_AREA)
    # Convert image to float32 and normalize pixel values
    normalized = np.float32(resized) / 255.0
    # Add batch dimension to image
    batched = np.expand_dims(normalized, axis=0)
    return batched

# Function to calculate facial condition scores
def calculate_scores(image):
    # Preprocess image for model input
    preprocessed_image = preprocess_image(image)
    # Use pre-trained model to predict facial condition scores
    score = model.predict(preprocessed_image)[0][0]
    # Calculate wrinkle score
    wrinkle_score = 1 - (1 / (1 + np.exp(-5*(score-50)/50)))
    # Calculate laxity score
    laxity_score = np.tanh(score / 10)
    # Calculate facial age
    facial_age = int(np.round(score / 10) * 10)
    # Calculate final score
    final_score = np.mean([wrinkle_score, laxity_score, facial_age])
    return wrinkle_score, laxity_score, facial_age, final_score

# Example usage
image = cv2.imread('20/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('30/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('40/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('50/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)

image = cv2.imread('60/1.jpg')
wrinkle_score, laxity_score, facial_age, final_score = calculate_scores(image)
print('Wrinkle score:', wrinkle_score)
print('Laxity score:', laxity_score)
print('Facial age:', facial_age)
print('Final score:', final_score)


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2316.9170
Epoch 2/100
1/1 [==============================] - 1s 583ms/step - loss: 2122.9338
Epoch 3/100
1/1 [==============================] - 1s 709ms/step - loss: 1737.5621
Epoch 4/100
1/1 [==============================] - 1s 591ms/step - loss: 1116.9745
Epoch 5/100
1/1 [==============================] - 1s 668ms/step - loss: 443.2067
Epoch 6/100
1/1 [==============================] - 1s 666ms/step - loss: 680.0927
Epoch 7/100
1/1 [==============================] - 0s 391ms/step - loss: 762.3412
Epoch 8/100
1/1 [==============================] - 0s 359ms/step - loss: 430.8914
Epoch 9/100
1/1 [==============================] - 0s 339ms/step - loss: 323.4872
Epoch 10/100
1/1 [==============================] - 0s 352ms/step - loss: 393.6266
Epoch 11/100
1/1 [==============================] - 0s 358ms/step - loss: 474.6310
Epoch 12/100
1/1 [==============================] - 0s 365ms/step - loss: 499.2517
Epoch 13/100